<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Summarization Using Vantage and LLM model
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction:</b></p>

<p style='font-size:16px;font-family:Arial'>In this demo we'll deep dive on Complaints Summarization using <b>Teradata Vantage</b> and <b>AWS Bedrock - Anthropic's Claude LLM model</b> model. This cutting-edge solution empowers organizations to efficiently manage and analyze customer complaints, providing actionable insights to enhance customer satisfaction and improve business operations.</p> 

<p style='font-size:16px;font-family:Arial'><b>Key Features:</b></p> 

<ol style='font-size:16px;font-family:Arial'>
  <li><b>AI-Powered Summarization</b>: Utilizing advanced natural language processing (NLP) and machine learning algorithms, the system automatically summarizes complaints, identifying key issues, sentiment, and root causes.</li>
  <li><b>Real-Time Analytics</b>: The platform provides real-time analytics and visualization tools, enabling users to track complaint trends, sentiment analysis, and issue resolution rates.</li>
  <li><b>Customizable Dashboards</b>: Users can create personalized dashboards to monitor specific complaint categories, product lines, or geographic regions, ensuring targeted insights and swift action.</li>
    <li><b>Integration with AWS Bedrock - Anthropic's Claude LLM model</b>: Seamless integration with <b>Teradata Vantage</b> and <b>AWS Bedrock - Anthropic's Claude LLM model</b> models enables users to leverage the power of cloud-based infrastructure and advanced analytics capabilities.</li>
</ol>

<p style='font-size:16px;font-family:Arial'><b>Benefits:</b></p> 
<ol style='font-size:16px;font-family:Arial'>
  <li><b>Enhanced Customer Experience</b>: By quickly identifying and addressing customer concerns, organizations can improve customer satisfaction and loyalty.</li>
  <li><b>Operational Efficiency</b>: Automated complaint summarization and analytics reduce manual processing time, allowing teams to focus on issue resolution and strategic decision-making.</li>
  <li><b>Data-Driven Decision-Making</b>: The platform provides actionable insights, enabling organizations to make informed decisions and drive business growth.</li>
</ol>

<p style = 'font-size:16px;font-family:Arial'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Configuring AWS Bedrock - Anthropic's Claude LLM model</li>
    <li>Complaints Summarization</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial'>1. Configuring the environment</b>
<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Downloading and installing additional software needed</b>

In [ ]:
%%capture
!pip install -r requirements.txt --upgrade --quiet

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.2 Import the required libraries</b></p>
<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Data manipulation and analysis
import numpy as np
import pandas as pd
import getpass

# Plotting
import plotly.express as px

# Progress bar
from tqdm import tqdm

# Machine learning and other utilities from Teradata
from teradataml import *
from teradatagenai import TeradataAI, TextAnalyticsAI, VSManager, VectorStore, VSApi

# Requests
import requests

# Display settings
display.max_rows = 5
pd.set_option('display.max_colwidth', None)

# Set display options for dataframes, plots, and warnings
%matplotlib inline
warnings.filterwarnings('ignore')
display.suppress_vantage_runtime_warnings = True

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
print("Checking if this environment is ready to connect to VantageCloud Lake...")

if os.path.exists("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env"):
    print("Your environment parameter file exist.  Please proceed with this use case.")
    # Load all the variables from the .env file into a dictionary
    env_vars = dotenv_values("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env")
    # Create the Context
    eng = create_context(host=env_vars.get("host"), username=env_vars.get("username"), password=env_vars.get("my_variable"))
    execute_sql('''SET query_band='DEMO=text_analytics_teradatagenai_aws_huggingface.ipynb;' UPDATE FOR SESSION;''')
    print("Connected to VantageCloud Lake with:", eng)
else:
    print("Your environment has not been prepared for connecting to VantageCloud Lake.")
    print("Please contact the support team.")

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>2.  Set up the LLM connection</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <b>teradatagenai</b> python library can both connect to cloud-based LLM services as well as instantiate private models running <b>at scale</b> on local GPU compute. In this case we will use anthropoc claude-instant-v1 for low-cost, high-throughput tasks.</p>

<ol style = 'font-size:16px;font-family:Arial'>
<li><b>aws_access_key_id</b>: Enter your AWS access key ID</li>
<li><b>aws_secret_access_key</b>: Enter your AWS secret access key</li>
<li><b>region name</b>: Enter the AWS region you want to configure (e.g., us-east-1)</li>
<ol>

In [ ]:
access_key = getpass.getpass('aws_access_key_id: ')
secret_key = getpass.getpass('aws_secret_access_key: ')
region_name = getpass.getpass('region name: ')

<hr style='height:2px;border:none'>
<p style='font-size:20px;font-family:Arial'><b>3. Use the <code>TextAnalyticsAI</code> API to Perform Various Text Analytics Tasks</b></p>
<p style='font-size:16px;font-family:Arial'>You can execute the help function at the bottom of this notebook to read more about this API.</p>

In [ ]:
# Provide model details
model_name="us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Select in-database or external model
llm = TeradataAI(api_type = 'AWS',
         model_name = model_name,
         region = region_name,
         # authorization = 'Repositories.BedrockAuth'
         access_key = access_key,
         secret_key = secret_key)

obj = TextAnalyticsAI(llm=llm)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Complaints summarization</b>
<p style = 'font-size:16px;font-family:Arial'>Complaints summarization with Language Model (LLM) models involves condensing lengthy complaints into concise, informative summaries. By leveraging advanced natural language processing techniques, LLMs efficiently extract key issues, sentiments, and resolutions, aiding in quicker understanding and response to customer grievances.</p>

<p style = 'font-size:16px;font-family:Arial'>Streamlining the complaint summarization process, Language Model (LLM) models efficiently distill verbose grievances into concise, yet informative summaries. These summaries meticulously capture crucial elements including primary issues, prevalent sentiments, and possible resolutions. Harnessing advanced natural language processing capabilities, LLMs accelerate both comprehension and response to customer concerns, thereby elevating operational efficiency and bolstering overall customer satisfaction.</p>

In [ ]:
df = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Consumer_Complaints'))

In [ ]:
tdf_summary = obj.summarize(column = 'consumer_complaint_narrative', 
                          data = df.iloc[:5],
                          levels = 2 # higher values provide more concise summary
                           )[['complaint_id','Summary','consumer_complaint_narrative']]

In [ ]:
tdf_summary

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.1 Graph for Complaint and Summary Lengths</b></p><p style = 'font-size:16px;font-family:Arial'>A graph illustrating the Narrative length vs summary length. On the x-axis, you'd have "Narrative length" ranging from short to long complaints or narratives. On the y-axis, you'd have "Summary length" ranging from brief to detailed summaries. As narrative length increases, summary length would generally decrease, indicating the summarization process effectively condenses longer narratives into shorter summaries. This relationship would likely follow a downward trend, showcasing the summarization efficiency of the LLM models.</p>

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# Truncate text for hover data
df = tdf_summary.to_pandas()
max_chars = 50  # Maximum characters to display
df['truncated_narrative'] = df['consumer_complaint_narrative'].apply(lambda x: x[:max_chars] + '...' if len(x) > max_chars else x)
df['truncated_summary'] = df['Summary'].apply(lambda x: x[:max_chars] + '...' if len(x) > max_chars else x)

# Calculate the length of consumer_complaint_narrative and Summary
df['narrative_length'] = df['consumer_complaint_narrative'].apply(len)
df['summary_length'] = df['Summary'].apply(len)

# Create a scatter plot
fig = px.scatter(df.sort_values(['narrative_length']), x='narrative_length', y='summary_length',
                 hover_data=['complaint_id', 'truncated_narrative', 'truncated_summary'],
                 labels={'narrative_length': 'Narrative Length', 'summary_length': 'Summary Length'},
                 title='Complaint and Summary Lengths')

# Update the x-axis to show values as they are (not in scientific notation)
fig.update_xaxes(type='category')

# Show the plot
fig.show()

<p style = 'font-size:16px;font-family:Arial'>Now the results can be saved back to Vantage.</p>

In [ ]:
copy_to_sql(df = df, table_name = 'Complaints_Summaries', if_exists = 'replace')

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>5. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ['Complaints_Summaries']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024, 2025, 2026. All Rights Reserved
        </div>
    </div>
</footer>